In [3]:
from pyspark import SparkContext
sc = SparkContext()

22/05/05 10:18:26 WARN Utils: Your hostname, Equipo resolves to a loopback address: 127.0.1.1; using 10.8.120.208 instead (on interface wlo1)
22/05/05 10:18:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/05 10:18:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
!head -1 201704_Usage_Bicimad.json

{ "_id" : { "$oid" : "58e5e5e7865e9c221ca45898" }, "user_day_code" : "df843a74bba22d2b363e4e96f74b99391276e71dfea92e0a11de4fe552f38173", "idplug_base" : 17, "user_type" : 1, "idunplug_base" : 1, "travel_time" : 169, "idunplug_station" : 41, "ageRange" : 4, "idplug_station" : 50, "unplug_hourTime" : { "$date" : "2017-04-01T01:00:00.000+0200" }, "zip_code" : "28005" }


In [24]:
import json
import datetime

In [37]:
def mapper(line):
    data = json.loads(line)
    usert = data['user_type']
    time = data['travel_time']
    date = convert_date(data['unplug_hourTime'])
    return usert, time, date

In [70]:
def convert_date(date):
    if isinstance(date, str):
        date = date.replace("Z","+0000")
        date = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%S%z")
    else:
        date = date['$date']
        date = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%S.%f%z")
    return (date.month,date.hour)

In [71]:
convert_date({ "$date" : "2019-06-01T12:40:00.000+0200" })

(6, 12)

In [67]:
rdd_base = sc.textFile('sample_10e3.json')

In [63]:
rdd = rdd_base.map(mapper)

In [56]:
rdd.countByKey()
#0: no ha sido posible determinarlo, 1: pase anual, 2: ocasional, 3: empresa

defaultdict(int, {1: 954, 2: 10, 3: 36})

In [72]:
rdd.groupByKey()
rdd.take(10)

[(1, 219, (6, 0)),
 (1, 359, (6, 0)),
 (1, 375, (6, 0)),
 (1, 264, (6, 0)),
 (1, 367, (6, 0)),
 (1, 174, (6, 0)),
 (1, 308, (6, 0)),
 (1, 462, (6, 0)),
 (1, 482, (6, 0)),
 (1, 480, (6, 0))]

In [58]:
rdd_user1 = rdd.filter(lambda x: x[0]==1).map(lambda x: (x[1],tuple(x[2:])))
rdd_user2 = rdd.filter(lambda x: x[0]==2).map(lambda x: (x[1],tuple(x[2:])))
rdd_user3 = rdd.filter(lambda x: x[0]==3).map(lambda x: (x[1],tuple(x[2:])))

In [74]:
rdd_user1.take(3)

[(219, ((6, 0),)), (359, ((6, 0),)), (375, ((6, 0),))]